<a href="https://colab.research.google.com/github/rahiakela/data-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/11-deep-learning-for-text/01_words_representing_approach_sets_and_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Words representing approache: Sets and sequences

A much more problematic question,
however, is how to encode the way words are woven into sentences: word order.

The problem of order in natural language is an interesting one: unlike the steps of a timeseries, words in a sentence don’t have a natural, canonical order.

The simplest thing you could do is just discard order and
treat text as an unordered set of words—this gives you **bag-of-words models**.

You could also decide that words should be processed strictly in the order in which they appear, one at a time, like steps in a timeseries—you could then leverage the **recurrent models**.

Finally, a hybrid approach is also possible: the **Transformer architecture** is technically order-agnostic, yet it injects word-position information into
the representations it processes, which enables it to simultaneously look at different parts of a sentence, while still being order-aware. 

Because they take into account word order, **both RNNs and Transformers are called sequence models.**

We’ll demonstrate each approach on a well-known text classification benchmark:
the IMDB movie review sentiment-classification dataset.

Let’s process the raw IMDB text data, just like you would do when approaching a new text-classification problem in the real world.





##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import os, pathlib, shutil, random

Let’s start by downloading the dataset from the Stanford page and uncompressing it:

In [7]:
%%shell

curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

# delete unwanted file and subdirectory
rm -rf aclImdb/train/unsup
rm -rf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8090k      0  0:00:10  0:00:10 --:--:-- 16.7M


In [10]:
# take a look at the content of a few of these text files
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

##Preparing the IMDB movie reviews data

For instance, the `train/pos/` directory contains a set of `12,500` text files, each of which
contains the text body of a positive-sentiment movie review to be used as training data.
The negative-sentiment reviews live in the “neg” directories. 

In total, there are `25,000`
text files for training and another 25,000 for testing.

Next, let’s prepare a validation set by setting apart 20% of the training text files in a new directory, `aclImdb/val`:

In [12]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/"val"
train_dir = base_dir/"train"

for category in ("neg", "pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code
  random.Random(1337).shuffle(files)
  # Take 20% of the training files to use for validation
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    # Move the files to aclImdb/val/neg and aclImdb/val/pos
    shutil.move(train_dir/category/fname, val_dir/category/fname)

Remember how, we used the `image_dataset_from_directory` utility to
create a batched Dataset of images and their labels for a directory structure? You can do the exact same thing for text files using the `text_dataset_from_directory` utility.

Let’s create three Dataset objects for training, validation, and testing:

In [13]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


These datasets yield inputs that are TensorFlow `tf.string` tensors and targets that are `int32` tensors encoding the value “0” or “1.”

In [14]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Cheesy script, cheesy one-liners. Timothy Hutton\'s performance a "little" over the top. David Duchovny still seemed to be stuck in his Fox Mulder mode. No chemistry with his large-lipped female co-star.He needs Gillian Anderson to shine. He does not seem to have any talent of his own.', shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


All set. Now let’s try learning something from this data.

##Processing words as a set: The bag-of-words approach